In [1]:
import heeps
from heeps.wavefront.get_array import get_wf
from heeps.util.psf_template import psf_template
from heeps.util.lamD import get_lamD_mas
import numpy as np
from astropy.io import fits

In [2]:
def get_band_mode(case):
    band_mode = {'ELT-L' :  ('L', 'ELT'),
                 'ELT-N' :  ('N2', 'ELT'),
                 'APP-LM' : ('L', 'APP'),
                 'RLS-LM' : ('L', 'RAVC'),
                 'CLS-LM' : ('L', 'CVC'),
                 'ULS-LM' : ('L', 'CLC'),
                 'CLS-N'  : ('N2', 'CVC'),
                 'ULS-N'  : ('N2', 'CLC'),
                 'APP-LMS': ('L', 'APP'),
                 'RLS-LMS': ('L', 'RAVC'),
                 'CLS-LMS': ('L', 'CVC'),
                 'ULS-LMS': ('L', 'CLC')}
    return band_mode[case]

def get_margins(case):
    ls_dR = {'ELT-L' :  (0, 0, 0),# no stop at all
             'ELT-N' :  (0, 0, 0),# no stop at all
             'APP-LM' : (2.09, 2.09, 2.91),# load fits file
             'RLS-LM' : (4.77, 4.00, 2.49),
             'CLS-LM' : (2.09, 9.00, 2.45),
             'ULS-LM' : (10.95, 4.91, 3.17),
             'CLS-N'  : (2.68, 9.00, 3.57),
             'ULS-N'  : (10.45, 5.11, 3.83),
             'APP-LMS': (2.41, 2.41, 3.08),# load fits file
             'RLS-LMS': (4.68, 4.00, 2.80),
             'CLS-LMS': (2.18, 9.00, 2.88),
             'ULS-LMS': (10.95, 4.91, 3.17)}
    return np.round(np.array(ls_dR[case])/100, 4)

def get_throughput(case, ap_flux_ELT=None, verbose=False):
    band, mode = get_band_mode(case)
    ls_dRext, ls_dRint, ls_dRspi = get_margins(case)
    det = 'LMS' if 'LMS' in case else 'IMG'
    f_app_amp = 'optics/app/METIS_%s_L_amp.fits'%det
    conf = dict(band=band, mode=mode, nframes=1, add_phase=False, f_app_amp=f_app_amp,
                ls_dRext=ls_dRext, ls_dRint=ls_dRint, ls_dRspi=ls_dRspi)
    conf = heeps.config.read_config(**conf)
    conf = heeps.config.update_config(**conf, verbose=verbose)
    wf = heeps.pupil.pupil(**conf)
    wf = heeps.optics.apodizer(wf, onaxis=False, **conf, verbose=verbose)
    wf = heeps.optics.lyot_stop(wf, **conf, verbose=verbose)
    pupil_throughtput = np.sum(get_wf(wf, 'amp')**2)
    print('pupil throughput (wrt ELT) = %.4f'%pupil_throughtput)
    psf_OFF = heeps.optics.detector(wf, **conf)
    _, fwhm, ap_flux = psf_template(psf_OFF)
    print('fwhm = %.2fpix, ap_flux = %.4f'%(fwhm, ap_flux))
    if 'ELT' in mode:
        return ap_flux
    print('FWHM throughput (wrt ELT) = %.4f'%(ap_flux/ap_flux_ELT))
    if 'VC' in mode:
        data = fits.getdata(conf['f_vc_trans'])
        mask_trans = np.interp(conf['lam']*1e6, data[0], data[1])
        print('VPM transmittance = %.4f'%mask_trans)
        oat = fits.getdata(conf['dir_input'] + 'optics/oat_%s_%s.fits'%(band, mode))
        npix = 5*get_lamD_mas(**conf)/conf['pscale']
        oat5 = np.interp(npix, oat[0], oat[1])
        print('OAT @ 5 lambda/D = %.4f'%oat5)
    elif 'APP' in mode:
        data = fits.getdata(conf['f_app_trans'])
        mask_trans = np.interp(conf['lam']*1e6, data[0], data[1])
        print('APP transmittance = %.4f'%mask_trans)
        print('APP Strehl ratio =', conf['app_strehl'])
        print('APP single PSF =', conf['app_single_psf'])
        oat5 = conf['app_strehl'] * conf['app_single_psf']
    else:
        mask_trans = 1
        oat5 = 1
    TOTAL =  ap_flux/ap_flux_ELT * mask_trans * oat5
    print('Total throughput = %.4f'%TOTAL)
    return TOTAL

### ELT aperture photometry

In [3]:
ap_flux_ELT_L = get_throughput('ELT-L')

pupil throughput (wrt ELT) = 1.0000
fwhm = 3.51pix, ap_flux = 0.3513


In [4]:
ap_flux_ELT_N = get_throughput('ELT-N')

pupil throughput (wrt ELT) = 1.0000
fwhm = 8.52pix, ap_flux = 0.3687


# LM modes

In [5]:
get_throughput('APP-LM', ap_flux_ELT_L);

pupil throughput (wrt ELT) = 0.8095
fwhm = 3.56pix, ap_flux = 0.2356
FWHM throughput (wrt ELT) = 0.6706
APP transmittance = 0.7334
APP Strehl ratio = 0.6
APP single PSF = 0.48
Total throughput = 0.1416


In [6]:
get_throughput('RLS-LM', ap_flux_ELT_L);

pupil throughput (wrt ELT) = 0.3668
fwhm = 3.30pix, ap_flux = 0.0679
FWHM throughput (wrt ELT) = 0.1931
VPM transmittance = 0.8532
OAT @ 5 lambda/D = 0.8829
Total throughput = 0.1455


In [7]:
get_throughput('CLS-LM', ap_flux_ELT_L);

pupil throughput (wrt ELT) = 0.7974
fwhm = 3.48pix, ap_flux = 0.2199
FWHM throughput (wrt ELT) = 0.6258
VPM transmittance = 0.8532
OAT @ 5 lambda/D = 0.8188
Total throughput = 0.4373


In [8]:
get_throughput('ULS-LM', ap_flux_ELT_L);

pupil throughput (wrt ELT) = 0.6340
fwhm = 3.85pix, ap_flux = 0.1684
FWHM throughput (wrt ELT) = 0.4794
Total throughput = 0.4794


# N modes

In [9]:
get_throughput('CLS-N', ap_flux_ELT_N);

pupil throughput (wrt ELT) = 0.7560
fwhm = 8.46pix, ap_flux = 0.2090
FWHM throughput (wrt ELT) = 0.5669
VPM transmittance = 0.9291
OAT @ 5 lambda/D = 0.8332
Total throughput = 0.4388


In [10]:
get_throughput('ULS-N', ap_flux_ELT_N);

pupil throughput (wrt ELT) = 0.6239
fwhm = 9.32pix, ap_flux = 0.1737
FWHM throughput (wrt ELT) = 0.4711
Total throughput = 0.4711


# LMS modes

In [11]:
get_throughput('APP-LMS', ap_flux_ELT_L);

pupil throughput (wrt ELT) = 0.8130
fwhm = 3.57pix, ap_flux = 0.2394
FWHM throughput (wrt ELT) = 0.6816
APP transmittance = 0.7334
APP Strehl ratio = 0.6
APP single PSF = 0.48
Total throughput = 0.1440


In [12]:
get_throughput('RLS-LMS', ap_flux_ELT_L);

pupil throughput (wrt ELT) = 0.3643
fwhm = 3.30pix, ap_flux = 0.0669
FWHM throughput (wrt ELT) = 0.1903
VPM transmittance = 0.8532
OAT @ 5 lambda/D = 0.8828
Total throughput = 0.1434


In [13]:
get_throughput('CLS-LMS', ap_flux_ELT_L);

pupil throughput (wrt ELT) = 0.7839
fwhm = 3.48pix, ap_flux = 0.2129
FWHM throughput (wrt ELT) = 0.6060
VPM transmittance = 0.8532
OAT @ 5 lambda/D = 0.8190
Total throughput = 0.4235


In [14]:
get_throughput('ULS-LMS', ap_flux_ELT_L); # same as ULC-LM

pupil throughput (wrt ELT) = 0.6340
fwhm = 3.85pix, ap_flux = 0.1684
FWHM throughput (wrt ELT) = 0.4794
Total throughput = 0.4794
